In [2]:

import os 
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers 
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
import matplotlib.pyplot as plt

In [4]:
#Fetch images Count from folders 
dirs = os.listdir('bones/')
for dir in dirs:
    files=list(os.listdir('bones/'+dir))
    print(dir)

fractured
not_fractured


In [5]:
#Fetch images Count from folders 
dirs = os.listdir('bones/')
for dir in dirs:
    files=list(os.listdir('bones/'+dir))
    print(dir + ' Folder has : ' +str(len(files)) + ' images')

fractured Folder has : 4480 images
not_fractured Folder has : 4383 images


In [9]:
#load Images into Arrays as Dataset
base_dir ='bones/'
img_size =180
batch =32

In [10]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    validation_split=0.2,
    subset="training",
    batch_size=batch,
    image_size=(img_size, img_size)
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,
    validation_split=0.2,
    subset="validation",
    batch_size=batch,
    image_size=(img_size, img_size)
)


Found 8863 files belonging to 2 classes.
Using 7091 files for training.
Found 8863 files belonging to 2 classes.
Using 1772 files for validation.


In [11]:
# Get class names
class_names = train_ds.class_names
print("Classes:", class_names)

Classes: ['fractured', 'not_fractured']


In [12]:
# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.prefetch(buffer_size=AUTOTUNE)

In [13]:
# Data augmentation layer
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [14]:

# Creating model
model = keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_size, img_size, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

C:\Users\hp\anaconda3\envs\NEURALNETWORK\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [16]:

# Model training
history = model.fit(train_ds, epochs=12, validation_data=validation_ds)

Epoch 1/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 459s 2s/step - accuracy: 0.5516 - loss: 0.6978 - val_accuracy: 0.6287 - val_loss: 0.6186
Epoch 2/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.6691 - loss: 0.6008 - val_accuracy: 0.7658 - val_loss: 0.4808
Epoch 3/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 314s 1s/step - accuracy: 0.7469 - loss: 0.4971 - val_accuracy: 0.8217 - val_loss: 0.3735
Epoch 4/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 313s 1s/step - accuracy: 0.8182 - loss: 0.3999 - val_accuracy: 0.8843 - val_loss: 0.2841
Epoch 5/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 4042s 18s/step - accuracy: 0.8602 - loss: 0.3208 - val_accuracy: 0.9018 - val_loss: 0.2218
Epoch 6/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 476s 2s/step - accuracy: 0.8918 - loss: 0.2538 - val_accuracy: 0.9565 - val_loss: 0.1224
Epoch 7/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.9314 - loss: 0.1805 - val_accuracy: 0.9684 - val_loss: 0.0911
Epoch 8/12
222/222 ━━━━━━━━━━━━━━━━━━━━ 332s 1s/step - accuracy: 0.9361 - loss: 0.1606 - val_ac

In [17]:
 
# Save model
model.save("bone_fracture_model.h5")
print("Model saved as bone_fracture_model.h5")

Model saved as bone_fracture_model.h5
